In [4]:
import csv
import nltk
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

## Loading model

In [5]:
print("loading Model . . . ")
model =   KeyedVectors.load_word2vec_format('../../../embedding/GoogleNews-vectors-negative300.bin', binary=True)
print("google w2v loaded")

loading Model . . . 
google w2v loaded


In [6]:
from gensim.models import Word2Vec
embeddings_index = {}
SO_file = 'textModel'
model2 = Word2Vec.load('../../../embedding/'+ SO_file)
print("SO w2v loaded")

SO w2v loaded


## Loading data

In [7]:
import _pickle as pk
preprocessed_data = "preprocessed_data_noClue_T+B_medium/"   
train_q1_Body= pk.load(open(preprocessed_data+"train_q1_Body.pkl", 'rb'))
train_q2_Body= pk.load(open(preprocessed_data+"train_q2_Body.pkl", 'rb'))
train_labels= pk.load(open(preprocessed_data+"train_class.pkl", 'rb'))

In [8]:
 
with open(preprocessed_data+"test_q1_Body.pkl", 'rb') as fb:
    test_q1_Body= pk.load(fb)

with open(preprocessed_data+"test_q2_Body.pkl", 'rb') as fb:
    test_q2_Body= pk.load(fb)
    
with open(preprocessed_data+"test_class.pkl", 'rb') as fb:
    test_labels= pk.load(fb)

## Features:

In [9]:
all_text_lst =  train_q1_Body + train_q2_Body +test_q1_Body  + test_q2_Body 

In [10]:

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=nltk.word_tokenize,
                                             analyzer="word", lowercase=True)
fittedTFIDFvectorizer = tfidf_vectorizer.fit_transform(all_text_lst)
print("MODEL fitted!")

MODEL fitted!


In [11]:

def common_bigrams( q1, q2, model):
    q1_words = nltk.word_tokenize(q1)
    q1BiGrams = list(nltk.bigrams(q1_words))
    q2_words = nltk.word_tokenize(q2)
    q2BiGrams = list(nltk.bigrams(q2_words))

    matches = 0
    for questionBigram in q1BiGrams:
        for parentBigram in q2BiGrams:
            if questionBigram == parentBigram:
                matches += 1
    return matches

In [12]:
def common_trigrams( q1, q2, model):
    q1_words = nltk.word_tokenize(q1)
    q1BiGrams = list(nltk.trigrams(q1_words))
    q2_words = nltk.word_tokenize(q2)
    q2BiGrams = list(nltk.trigrams(q2_words))

    matches = 0
    for questionBigram in q1BiGrams:
        for parentBigram in q2BiGrams:
            if questionBigram == parentBigram:
                matches += 1
    return matches

In [13]:
def common_char_trigrams( q1, q2, model):
    char_q1 = [q1[i:i+3] for i in range(len(q1)-1)]
    char_q2 = [q2[i:i+3] for i in range(len(q2)-1)]
    matches = 0
    for c1 in char_q1:
        for c2 in char_q2:
            if c1==c2:
                matches+=1
    return matches
                
    

In [14]:
def common_char_fourgrams( q1, q2, model):
    char_q1 = [q1[i:i+4] for i in range(len(q1)-1)]
    char_q2 = [q2[i:i+4] for i in range(len(q2)-1)]
    matches = 0
    for c1 in char_q1:
        for c2 in char_q2:
            if c1==c2:
                matches+=1
    return matches

In [15]:
def common_char_fivegrams( q1, q2, model):
    char_q1 = [q1[i:i+5] for i in range(len(q1)-1)]
    char_q2 = [q2[i:i+5] for i in range(len(q2)-1)]
    matches = 0
    for c1 in char_q1:
        for c2 in char_q2:
            if c1==c2:
                matches+=1
    return matches

In [16]:
def common_unigrams( q1, q2, model):
    q1_words =nltk.word_tokenize(q1)
    q2_words = nltk.word_tokenize(q2)
    matches = 0
    for q1 in q1_words:
        for q2 in q2_words:
            if q2 == q1:
                matches += 1
    return matches

In [17]:
print(common_unigrams("hello ","hello", model ))

1


In [18]:
import math
def soft_w2v(q1, q2, model,fittedTFIDF):
    if len(q1)==0 or len(q2)==0: return 0
    matrix0, matrix1 ,featureNames = vectorizer(q1 , q2, fittedTFIDF)
    index = []
    for i in np.nonzero(matrix0)[0]:
        index.append(i)
    for i in np.nonzero(matrix1)[0]:
        index.append(i)
    index = sorted(set(index))
    new_matrix0 = []
    new_matrix1 = []
    for l in index:
        new_matrix0.append(matrix0[l])
        new_matrix1.append(matrix1[l])
    docs = []
    docs.append(q1)
    docs.append(q2)
    featureNameLSZT = []
    for g in index:
        featureNameLSZT.append(featureNames[g])
    matrix = matrixByw2v(featureNameLSZT, model)
    npM0 = np.asarray(new_matrix0)
    npM1 = np.asarray(new_matrix1)
    tnpM0 = npM0.transpose()
    tnpM1 = npM1.transpose()
    dotProduct1 = np.dot(tnpM0, matrix)
    dotproduct2 = np.dot(dotProduct1, npM1)
    firstDomPartOne = np.dot(tnpM0, matrix)
    firstDomPartTwo = np.dot(firstDomPartOne, npM0)
    firstDominator = math.sqrt(firstDomPartTwo)
    secondDomPartOne = np.dot(tnpM1, matrix)
    secondDomPartTwo = np.dot(secondDomPartOne, npM1)
    secondDominator = math.sqrt(secondDomPartTwo)
    softCosineSimilarity = dotproduct2 / (firstDominator * secondDominator)
    if np.count_nonzero(new_matrix0)==0 or np.count_nonzero(new_matrix1)==0:
        return 0
    return softCosineSimilarity

def matrixByw2v(featureNames, model):

    matrix = np.zeros((len(featureNames), len(featureNames))).astype('float')
    # matrix =[[]]
    for i in range(len(featureNames)):
        for j in range(len(featureNames)):
            if i == j:
                matrix[i][j] = 1
            else:
                try:
                    sim = model.similarity(featureNames[i], featureNames[j])
                except KeyError:
                    sim = 0
                matrix[i][j] = (max(0, sim) * max(0, sim))
    return matrix

def vectorizer(question, parentQuestion, fittedTFIDF):
    if len(question) == 0 or len(parentQuestion) == 0:
        return [0]
    documents = []
    documents.append(question)
    documents.append(parentQuestion)
    tfidf_matrix = fittedTFIDF.transform(documents).toarray()
    featureNames = fittedTFIDF.get_feature_names()
    return tfidf_matrix[0], tfidf_matrix[1] ,featureNames


In [19]:
import Levenshtein
def soft_lev(q1, q2, model,fittedTFIDF):
    if len(q1)==0 or len(q2)==0: return 0
    matrix0, matrix1 ,featureNames = vectorizer(q1 , q2, fittedTFIDF)
    index = []
    for i in np.nonzero(matrix0)[0]:
        index.append(i)
    for i in np.nonzero(matrix1)[0]:
        index.append(i)
    index = sorted(set(index))
    new_matrix0 = []
    new_matrix1 = []
    for l in index:
        new_matrix0.append(matrix0[l])
        new_matrix1.append(matrix1[l])
    docs = []
    docs.append(q1)
    docs.append(q2)
    featureNameLSZT = []
    for g in index:
        featureNameLSZT.append(featureNames[g])
    matrix = matrixByLev(featureNameLSZT, model)
    npM0 = np.asarray(new_matrix0)
    npM1 = np.asarray(new_matrix1)
    tnpM0 = npM0.transpose()
    tnpM1 = npM1.transpose()
    dotProduct1 = np.dot(tnpM0, matrix)
    dotproduct2 = np.dot(dotProduct1, npM1)
    firstDomPartOne = np.dot(tnpM0, matrix)
    firstDomPartTwo = np.dot(firstDomPartOne, npM0)
    firstDominator = math.sqrt(firstDomPartTwo)
    secondDomPartOne = np.dot(tnpM1, matrix)
    secondDomPartTwo = np.dot(secondDomPartOne, npM1)
    secondDominator = math.sqrt(secondDomPartTwo)
    softCosineSimilarity = dotproduct2 / (firstDominator * secondDominator)
    if np.count_nonzero(new_matrix0)==0 or np.count_nonzero(new_matrix1)==0:
        return 0
    return softCosineSimilarity

def matrixByLev(featureNames, model):

    matrix = np.zeros((len(featureNames), len(featureNames))).astype('float')
    # matrix =[[]]
    for i in range(len(featureNames)):
        for j in range(len(featureNames)):

            if i == j:
                matrix[i][j] = 1
            else:
                # print("similarity of word {0} and {1}".format(featureNames[i], featureNames[j]))
                try:
                    lev = Levenshtein.distance(featureNames[i], featureNames[j])
                    lenfirst = len(featureNames[i])
                    lensecond = len(featureNames[j])
                    matrix[i][j] = 1.8 * (math.pow(1 - lev / max(lenfirst, lensecond), 5))
                except KeyError:
                    print("error in edit distance")


    return matrix



In [20]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_similarity(q1, q2, model,fittedTFIDF):
    
    
    if len(q1)==0 or len(q2) ==0:
        return 0
    documents =[]
    documents.append(q1)
    documents.append(q2)
    tfidf_matrix = fittedTFIDF.transform(documents)
    matrix = cosine_similarity(tfidf_matrix[0,:], tfidf_matrix[1,:])
    return float(matrix[0][0])

In [21]:
print(cos_similarity("hi hello hell","hello hell hi",model,tfidf_vectorizer ))

1.0


In [22]:



train_bigrams =[]
train_unigrams =[]
train_trigrams =[]
train_char_three = []
train_char_four =[]
train_char_five =[]
soft_train_google =[]
soft_train_SO =[]
soft_train_lev = []

cos_train_body =[]

test_bigrams =[]
test_unigrams =[]
test_trigrams =[]
test_char_three = []
test_char_four =[]
test_char_five =[]
soft_test_google =[]
soft_test_SO =[]
soft_test_lev = []

cos_test_body =[]
        
    


In [23]:
print(len(train_q1_Body))
for i in range(len(train_q1_Body)):
    
        postText = train_q1_Body[i] 
        relatedPostText =  train_q2_Body[i] 

        train_char_five.append(common_char_fivegrams(postText, relatedPostText, model))
        train_char_four.append(common_char_fourgrams(postText, relatedPostText, model))
        train_char_three.append(common_char_trigrams(postText, relatedPostText, model))
        train_trigrams.append(common_trigrams(postText, relatedPostText, model))

        train_bigrams.append(common_bigrams(postText, relatedPostText, model))
        train_unigrams.append(common_unigrams(postText, relatedPostText, model))

        softw2v1 =soft_w2v(postText, relatedPostText, model, tfidf_vectorizer)

        soft_train_google.append(softw2v1)

        softw2v2 = soft_w2v(postText, relatedPostText, model2, tfidf_vectorizer)
        soft_train_SO.append(softw2v2)

        softlev = soft_lev(postText, relatedPostText, model2, tfidf_vectorizer)
        soft_train_lev.append(softlev)
        cos_train_body.append(cos_similarity(postText, relatedPostText, model, tfidf_vectorizer))

32000


/share/apps/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/share/apps/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [24]:
print(len(cos_train_body))
# new_cos=[]
# for i in cos_train_body:
#     if type(i)==list:
#         new_cos.append(i[0])
#     else:
#         new_cos.append(i)
# print(len(new_cos))
np_cos_train_body= np.array(cos_train_body)

32000


In [25]:
#train:
print(len(train_q1_Body))

np_train_char_five = np.array(train_char_five)
print(len(np_train_char_five))
np_train_char_four= np.array(train_char_four)
np_train_char_three= np.array(train_char_three)
print(len(train_char_three))
np_train_trigrams= np.array(train_trigrams)

np_train_bigrams = np.array(train_bigrams)
print(len(np_train_bigrams))
np_train_unigrams = np.array(train_unigrams)
print(len(np_train_unigrams))
print("np_train_unigrams: ", np_train_unigrams[:3])
np_soft_train_google = np.array(soft_train_google)
print(len(np_soft_train_google))
np_soft_train_SO = np.array(soft_train_SO)
print(len(np_soft_train_SO))
np_soft_train_lev = np.array(soft_train_lev)
print(len(soft_train_lev))


train_all_feats = np.stack((np_train_char_five, np_train_char_four, np_train_char_three ,np_train_trigrams, np_train_bigrams, np_train_unigrams,
                            np_soft_train_SO, np_soft_train_google, np_soft_train_lev, np_cos_train_body), axis=-1)
print("all_feats: ", train_all_feats[:3])
print(np_train_bigrams.size)
print(np_train_unigrams.size)
print(train_all_feats.shape)
np.save(open('hand-feats/train_all_feats_noClue_medium.npy', 'wb'), train_all_feats)

32000
32000
32000
32000
32000
np_train_unigrams:  [24 13 33]
32000
32000
32000
all_feats:  [[1.05000000e+02 1.38000000e+02 1.99000000e+02 2.00000000e+00
  5.00000000e+00 2.40000000e+01 4.79051750e-01 5.33337488e-01
  3.80178128e-01 3.55610304e-01]
 [6.40000000e+01 9.40000000e+01 1.53000000e+02 0.00000000e+00
  0.00000000e+00 1.30000000e+01 3.33382290e-01 3.76200833e-01
  3.51270902e-01 2.18593091e-01]
 [1.36000000e+02 1.78000000e+02 2.90000000e+02 4.00000000e+00
  1.60000000e+01 3.30000000e+01 3.77879165e-01 4.49246605e-01
  3.45845517e-01 3.21577313e-01]]
32000
32000
(32000, 10)


In [26]:
print(len(test_q1_Body))
for i in range(len(test_q1_Body)):
        
    postText = test_q1_Body[i] 
    relatedPostText =  test_q2_Body[i]
    
    test_char_five.append(common_char_fivegrams(postText, relatedPostText, model))
    test_char_four.append(common_char_fourgrams(postText, relatedPostText, model))
    test_char_three.append(common_char_trigrams(postText, relatedPostText, model))
    test_trigrams.append(common_trigrams(postText, relatedPostText, model))
        
    test_bigrams.append(common_bigrams(postText, relatedPostText, model))
    test_unigrams.append(common_unigrams(postText, relatedPostText, model))
    softw2v1 =soft_w2v(postText, relatedPostText, model, tfidf_vectorizer)

    soft_test_google.append(softw2v1)

    softw2v2=soft_w2v(postText, relatedPostText, model2, tfidf_vectorizer)

    soft_test_SO.append(softw2v2)

    softlev = soft_lev(postText, relatedPostText, model2, tfidf_vectorizer)

    soft_test_lev.append(softlev)
    if (len(test_char_five)!=len(soft_test_lev)):
        print(i)
    cos_test_body.append(cos_similarity(postText, relatedPostText, model, tfidf_vectorizer))


8000


/share/apps/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/share/apps/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [27]:
print(len(cos_test_body))

# new_cos=[]
# for i in cos_test_body:
#     if type(i)==list:
#         new_cos.append(i[0])
#     else:
#         new_cos.append(i)
# print(len(new_cos))
np_cos_test_body= np.array(cos_test_body)

8000


In [28]:
#test:
np_test_char_five = np.array(test_char_five)
print(len(np_test_char_five))
np_test_char_four= np.array(test_char_four)
print(len(np_test_char_four))
np_test_char_three= np.array(test_char_three)
print(len(np_test_char_three))
np_test_trigrams= np.array(test_trigrams)
print(len(np_test_trigrams))
np_test_bigrams = np.array(test_bigrams)
print("np_test_bigrams: ", len(np_test_bigrams))
np_test_unigrams = np.array(test_unigrams)
print("np_test_unigrams: ", len(np_test_unigrams))
np_soft_test_google = np.array(soft_test_google)
print("google: ",len(np_soft_test_google))
np_soft_test_SO = np.array(soft_test_SO)
print("so: ",len(np_soft_test_SO))
np_soft_test_lev = np.array(soft_test_lev)
print(len(np_soft_test_lev))
#np_cos_test_body= np.array(cos_test_body)
# 
test_all_feats = np.stack((np_test_char_five,np_test_char_four,np_test_char_three,np_test_trigrams,np_test_bigrams, np_test_unigrams, np_soft_test_SO, np_soft_test_google, np_soft_test_lev, np_cos_test_body), axis=-1)
print("all_feats: ", test_all_feats[:3])
print(np_test_bigrams.size)
print(np_test_unigrams.size)
print(test_all_feats.shape)
np.save(open('hand-feats/test_all_feats_noClue_medium.npy', 'wb'), test_all_feats)

8000
8000
8000
8000
np_test_bigrams:  8000
np_test_unigrams:  8000
google:  8000
so:  8000
8000
all_feats:  [[3.36000000e+02 4.30000000e+02 6.13000000e+02 1.10000000e+01
  1.60000000e+01 4.60000000e+01 7.51451734e-01 7.71761344e-01
  7.33846546e-01 6.78276176e-01]
 [4.10000000e+01 6.50000000e+01 1.44000000e+02 0.00000000e+00
  0.00000000e+00 1.10000000e+01 1.37659936e-01 2.52587077e-01
  8.58362485e-02 5.49876580e-02]
 [6.20000000e+01 1.42000000e+02 3.22000000e+02 0.00000000e+00
  0.00000000e+00 1.90000000e+01 2.19694698e-01 3.48257934e-01
  1.60015982e-01 6.29924873e-02]]
8000
8000
(8000, 10)
